In [1]:
import numpy as np
data = np.random.randint(0, 10, (100, 5)).tolist()
distData = sc.parallelize(data)
print(type(data[0][0]))

## Od tąd zaczyna się moja część - poprzednia komórka to wygenerowanie mockowych danych - wektorów, których oczkuję od cześci Maćka.

In [3]:
from pyspark.sql import Row
from pyspark.sql.types import * 
from pyspark.ml.linalg import Vectors

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Generowanie losowych labels
randomLabels = sc.parallelize(np.random.randint(0, 3, distData.count(), dtype=int).tolist())

# Tworzenie DF-a z labels i features
dataWithLabels = distData.zip(randomLabels)
dataWithLabelsDF = dataWithLabels.map(lambda row: Row(label=row[1], features=Vectors.dense(row[0]))).toDF()

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

# train test split - do testów
training, test = dataWithLabelsDF.randomSplit([0.8, 0.2])

# Uzywam LogisticRegression do klasyfikacji
lr = LogisticRegression(maxIter=5, regParam=0.01)

# Param grid do CrossValidation - pewnie będzie rozbudowany o transformery
grid = ParamGridBuilder() \
.addGrid(lr.maxIter, [5, 10]) \
.addGrid(lr.regParam, [0.01, 0.1]) \
.build()
cv = CrossValidator(estimator=lr, estimatorParamMaps=grid, evaluator=evaluator)

# Trening
model = cv.fit(training)

# Testy na zbiorze testowym
results = model.transform(test)

# Ewaluacja wyników - żałosna skuteczność przez losowość klas.
evaluator.evaluate(results)